In [ ]:
## Acknowledgement

This notebook is from diffusers [release page](https://github.com/huggingface/diffusers/releases/tag/v0.20.0). [Colab Link](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/SDXL_DreamBooth_LoRA_.ipynb).

In [ ]:
!nvidia-smi

In [ ]:
!pip install torch torchvision torchdata torchtext torchaudio -U -q

In [ ]:
!pip install xformers bitsandbytes transformers accelerate -q

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git -q

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

In [ ]:
import torch, diffusers, transformers, accelerate, bitsandbytes

(
    torch.__version__, diffusers.__version__, 
    transformers.__version__, accelerate.__version__
)

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols, resize=256):
    assert len(imgs) == rows * cols

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
import glob

imgs = [Image.open(path) for path in glob.glob("./dog/*.jpeg")]
image_grid(imgs, 1, 5)

In [ ]:
!accelerate config default

In [ ]:
#!/usr/bin/env bash
command = (
    f'accelerate launch train_dreambooth_lora_sdxl.py '
    f'--pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" '
    f'--pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" '
    f'--instance_data_dir="dog" '
    f'--output_dir="lora-trained-xl-colab" '
    f'--mixed_precision="fp16" '
    f'--instance_prompt="a photo of sks dog" '
    f'--resolution=512 '
    f'--train_batch_size=1 '
    f'--gradient_accumulation_steps=2 '
    f'--gradient_checkpointing '
    f'--learning_rate=1e-4 '
    f'--lr_scheduler="constant" '
    f'--lr_warmup_steps=0 '
    f'--enable_xformers_memory_efficient_attention '
    f'--use_8bit_adam '
    f'--max_train_steps=200 '
    f'--checkpointing_steps=717 '
    f'--seed="0" '
)

In [ ]:
!{command}

In [ ]:
from diffusers import DiffusionPipeline
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae, torch_dtype=torch.float16, variant="fp16",
    use_safetensors=True
)
pipe.load_lora_weights("sayakpaul/lora-trained-xl-colab")
_ = pipe.to("cuda")

In [ ]:
prompt = "a photo of sks dog with a cat"

image = pipe(prompt=prompt, num_inference_steps=25).images[0]
image